In [9]:
# Model is intended for n-charge same sign systems constrained to a sphere's surface
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize

R = 1  # Radius of sphere

def potential_energy(coords, q_vals):
    n = len(q_vals)
    theta = coords[:n]
    phi = coords[n:]

    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    U = 0
    epsilon = 1e-8  # Prevents dividing by zero
    for i in range(n):
        for j in range(i + 1, n):
            dx = x[i] - x[j]
            dy = y[i] - y[j]
            dz = z[i] - z[j]
            r = np.sqrt(dx**2 + dy**2 + dz**2) + epsilon
            U += q_vals[i] * q_vals[j] / r
    return U

def spherical_to_cartesian(theta, phi):
    x = R * np.sin(theta) * np.cos(phi)
    y = R * np.sin(theta) * np.sin(phi)
    z = R * np.cos(theta)
    return x, y, z

def compute_angles(x, y, z):
    n = len(x)
    angles = []
    for i in range(n):
        for j in range(i + 1, n):
            dot = x[i]*x[j] + y[i]*y[j] + z[i]*z[j]
            dot = np.clip(dot, -1.0, 1.0)
            angle = np.degrees(np.arccos(dot))
            angles.append((i+1, j+1, angle))
    return angles

def plot_charges(x, y, z):
    fig = go.Figure()

    # Draw the sphere
    u, v = np.mgrid[0:np.pi:40j, 0:2*np.pi:40j]
    xs = R * np.sin(u) * np.cos(v)
    ys = R * np.sin(u) * np.sin(v)
    zs = R * np.cos(u)
    
    # Add the sphere
    fig.add_trace(go.Surface(
        x=xs, y=ys, z=zs, colorscale='gray', opacity=0.2
    ))

    # Add the charges
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z, mode='markers',
        marker=dict(size=15, color='red')  # Increase size of the charge dots
    ))

    # Add black lines from the center to each charge
    for i in range(len(x)):
        fig.add_trace(go.Scatter3d(
            x=[0, x[i]], y=[0, y[i]], z=[0, z[i]], 
            mode='lines',
            line=dict(color='black', width=3)  # width
        ))

    # Set plot layout for interactive features
    fig.update_layout(
        scene=dict(
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),  # Hide grid and labels
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            zaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            bgcolor='rgba(0,0,0,0)',  # Set the scene background to transparent
        ),
        title="Point Charges on Sphere",
        margin=dict(l=0, r=0, b=0, t=40),
        plot_bgcolor='rgba(0,0,0,0)',  # Set the plot background to transparent
        paper_bgcolor='rgba(0,0,0,0)',  # Set the area outside the plot to transparent
        autosize=True, 
        showlegend=False,
    )

    fig.show()

if __name__ == "__main__":
    try:
        n = int(input("Enter the number of point charges (n ≥ 2): "))
        if n < 2:
            raise ValueError("Need at least 2 charges.")

        q_vals = []
        for i in range(n):
            val = float(input(f"Enter value for q{i+1}: "))
            q_vals.append(val)

        # Initial guess
        np.random.seed(0)
        
        # Fix the coordinates for q1 (theta_1 = 0, phi_1 = 0)
        theta_guess = np.zeros(n)
        phi_guess = np.zeros(n)
        if n > 1:
            # For the remaining charges, add a small random perturbation
            theta_guess[1:] = np.linspace(0.2, np.pi - 0.2, n - 1) + np.random.normal(0, 0.05, n - 1)
            phi_guess[1:] = np.linspace(0, 2*np.pi, n - 1, endpoint=False) + np.random.normal(0, 0.1, n - 1)
        
        # Concatenate the guesses for optimization
        initial_guess = np.concatenate([theta_guess, phi_guess])

        # Bounds for theta and phi
        bounds = [(0, np.pi)] + [(0, 2 * np.pi)] + [(0, np.pi)] * (n - 1) + [(0, 2 * np.pi)] * (n - 1)

        # Minimize the potential energy
        result = minimize(potential_energy, initial_guess, args=(q_vals,),
                          method='L-BFGS-B', bounds=bounds)

        if not result.success:
            raise RuntimeError(f"Optimization failed: {result.message}")

        theta_opt = result.x[:n]
        phi_opt = result.x[n:]
        x, y, z = spherical_to_cartesian(theta_opt, phi_opt)

        # Results
        print("\nOptimized angles:")
        for i in range(n):
            print(f"θ{i+1} = {theta_opt[i]:.6f}, φ{i+1} = {phi_opt[i]:.6f}")

        print("\nAngles between each pair of charges:")
        angles = compute_angles(x, y, z)
        for i, j, angle in angles:
            print(f"Angle between charge {i} and {j}: {angle:.2f}°")

        plot_charges(x, y, z)

    except Exception as e:
        print(f"\nError: {e}")


Enter the number of point charges (n ≥ 2): 3
Enter value for q1: 1
Enter value for q2: 100
Enter value for q3: 21

Optimized angles:
θ1 = 0.000000, φ1 = 0.000000
θ2 = 2.105015, φ2 = 0.160977
θ3 = 1.099048, φ3 = 3.302577

Angles between each pair of charges:
Angle between charge 1 and 2: 120.61°
Angle between charge 1 and 3: 62.97°
Angle between charge 2 and 3: 176.42°
